In [3]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [4]:
%load_ext autoreload
%autoreload
from IPython.display import clear_output

import os
import pdb
import _pickle as cPickle
import time
import subprocess
import glob

import random
import numpy as np
import pandas as pd
import tensorflow as tf

from collections import defaultdict, Counter
from ncrp import Topic, Doc, init, sample, get_perplexity, get_topic_specialization, get_hierarchical_affinities, get_freq_tokens_ncrp, get_docs
from configure import get_config

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# load config & data 

In [5]:
config = get_config(nb_name)
np.random.seed(config.seed)
random.seed(config.seed)

In [6]:
instances_train, instances_dev, instances_test, word_to_idx, idx_to_word, bow_idxs = cPickle.load(open(config.path_data,'rb'))

In [7]:
config.n_doc = len(instances_train)
config.n_vocab = len(bow_idxs)
config.n_doc, config.n_vocab

(31943, 1035)

# run

## initialize log

In [8]:
checkpoint = []
ppl_min = np.inf
epoch = 0

cmd_rm = 'rm -r %s' % config.dir_model
res = subprocess.call(cmd_rm.split())
cmd_mk = 'mkdir %s' % config.dir_model
res = subprocess.call(cmd_mk.split())

log_df = pd.DataFrame(columns=pd.MultiIndex.from_tuples(
                    list(zip(*[['','','','TRAIN:','VALID:','TEST:','SPEC:', '', '', 'HIER:', ''],
                            ['Time','Ep','Ct','PPL','PPL', 'PPL','1', '2', '3', 'CHILD', 'OTHER']]))))

def update_checkpoint(config, checkpoint, epoch):
    checkpoint.append(config.path_model + '-%i' % epoch)
    if len(checkpoint) > config.max_to_keep:
        path_model = checkpoint.pop(0)
        for p in glob.glob(path_model):
            os.remove(p)
    cPickle.dump(checkpoint, open(config.path_checkpoint, 'wb'))

## initialize data

In [9]:
topic_root = Topic(idx='0', sibling_idx=0, parent=None, depth=0, config=config)
train_docs = get_docs(instances_train, config)
dev_docs = get_docs(instances_dev, config)
test_docs = get_docs(instances_test, config)
init(train_docs, dev_docs, test_docs, topic_root)

0 10000 20000 30000 0 0 

## run

In [ ]:
time_start = time.time()
while epoch < config.n_epochs:
    sample(train_docs, dev_docs, test_docs, topic_root)
    ppl_train = get_perplexity(train_docs, topic_root)
    ppl_dev = get_perplexity(dev_docs, topic_root)
    
    if ppl_dev < ppl_min:
        ppl_min = ppl_dev
        ppl_test = get_perplexity(test_docs, topic_root)
        cPickle.dump([test_docs, topic_root], open(config.path_model + '-%i'%epoch, 'wb'))
        update_checkpoint(config, checkpoint, epoch)
        
    depth_spec = get_topic_specialization(test_docs, topic_root)
    hierarchical_affinities = get_hierarchical_affinities(topic_root)
    
    clear_output()
    time_log = int(time.time() - time_start)
    time_start = time.time()
    log_series = pd.Series([time_log, epoch, 0, \
            '%.0f'%ppl_train, '%.0f'%ppl_dev, '%.0f'%ppl_test, \
            '%.2f'%depth_spec[1], '%.2f'%depth_spec[2], '%.2f'%depth_spec[3], \
            '%.2f'%hierarchical_affinities[0], '%.2f'%hierarchical_affinities[1]],
            index=log_df.columns)
    log_df.loc[epoch] = log_series    
    display(log_df)
    get_freq_tokens_ncrp(topic_root, idx_to_word, bow_idxs)
    
    cPickle.dump(log_df, open(config.path_log, 'wb'))
    epoch += 1

TRAIN: VALID: TEST: SPEC:             HIER:      
  Time Ep Ct    PPL    PPL   PPL     1     2     3 CHILD OTHER
0  260  0  0    344    331   329  0.07  0.56  0.60  0.48  0.33
1  328  1  0    308    303   304  0.08  0.59  0.63  0.50  0.28
2  354  2  0    302    302   303  0.09  0.61  0.63  0.48  0.24
3  351  3  0    302    303   303  0.09  0.64  0.63  0.49  0.23
4  377  4  0    295    294   295  0.10  0.65  0.63  0.47  0.23
5  403  5  0    294    297   295  0.10  0.65  0.61  0.49  0.25
6  443  6  0    288    290   291  0.10  0.66  0.62  0.47  0.24
7  436  7  0    284    283   285  0.10  0.65  0.61  0.50  0.25

 0 31943 301835.0 ! nice bought love price quality recommend perfect 'm made
   0-1 1527 8815.0 ! cover color love keyboard perfectly pro easy apple mac
     0-1-2 1522 3795.0 ! love cover mac & protector purple keyboard pink screen
     0-1-3 5 12.0 review scratching earlier pretty iphone move happen bill flap exact
   0-2 758 4556.0 carry room pockets items security airport plenty travel lots compartments
     0-2-1 3 5.0 velcro rough office sort traveling fingerprints finding fine finger zips
     0-2-2 755 1891.0 easy travel seat phone compartments files plane bottle overhead stuff
   0-3 641 4381.0 zipper broke strap months handle zippers broken return year started
     0-3-2 625 1734.0 months weeks broke shoulder ripped coming straps started strap wheels
     0-3-3 16 32.0 lost recently material cracked wait mention update minor months size
   0-4 1132 6153.0 cover mac keyboard scratches protects protect color easy pro apple
     0-4-1 39 86.0 mbp heavier unibody file knew point 